In [1]:
# -*- coding: utf8 -*-
from os.path import expanduser
home = expanduser("~")
annee="17"
nKalaba="K4"
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/"+annee+"-%s"%nKalaba
serie=repertoire+"/"
complementPhrases=""
phrasesFichiers=["Phrases","Ordre","Traductions"]
#phrasesFichiers=["Phrases"]
nomComptes=serie+complementPhrases+"Comptes.csv"
nomLexique=serie+"Lexique-S2.pkl"
debug=False

In [2]:
#########################IMPORTS############################################
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml, warnings
import ParFuMor as PFM
from ParFuMor import *
import pickle
import pandas as pd

In [3]:
with open(nomLexique, 'rb') as input:
   PFM.lexique = pickle.load(input)

In [4]:
with open(serie+"MorphoSyntax.yaml", 'r') as stream:
    morphosyntax=yaml.load(stream)
contractions=morphosyntax["Contractions"]

In [5]:
def compterLexique(exemplesPhrases):
    debug=0
    lexiquePhrases={}
    lexiqueComptes={}
    for nExemple,exemple in enumerate(exemplesPhrases):
        nPhrase=nExemple+1
        motsExemple=[mot for mot in re.split(ur"[\s',]",exemple) if mot!=""]
        for mot in motsExemple:
            if mot in PFM.lexique.formeLexeme:
                lexeme=PFM.lexique.formeLexeme[mot][0]
                if not lexeme in lexiqueComptes: lexiqueComptes[lexeme]=0
                lexiqueComptes[lexeme]+=1
                if not lexeme in lexiquePhrases: lexiquePhrases[lexeme]=set()
                lexiquePhrases[lexeme].add(nPhrase)
                if debug: print lexeme,
            elif mot in contractions:
                for element in contractions[mot]:
                    lexeme=PFM.lexique.formeLexeme[element][0]
                    if not lexeme in lexiqueComptes: lexiqueComptes[lexeme]=0
                    lexiqueComptes[lexeme]+=1
                    if not lexeme in lexiquePhrases: lexiquePhrases[lexeme]=set()
                    lexiquePhrases[lexeme].add(nPhrase)
                    if debug: print lexeme,
            else:
                lexeme=u"#%s#"%mot
                if not lexeme in lexiqueComptes: lexiqueComptes[lexeme]=0
                lexiqueComptes[lexeme]+=1
                if not lexeme in lexiquePhrases: lexiquePhrases[lexeme]=set()
                lexiquePhrases[lexeme].add(nPhrase)
                if debug: print lexeme,
        if debug: print
    return lexiqueComptes,lexiquePhrases

In [6]:
def stripLex(lex):
    return lex.strip("#")

In [7]:
def printLexPhrases(lexPhrases):
    for lex in sorted(lexPhrases.keys()):
        print stripLex(lex.split(".")[0]),", ".join(["(%d)"%num for num in sorted(lexPhrases[lex])])

In [8]:
def printLexComptes(lexComptes):
    for lex in sorted(lexComptes, key=lexComptes.get,reverse=True):
        print stripLex(lex.split(".")[0]), lexComptes[lex]

In [9]:
dfLexComptes=[]

In [10]:
for nFichier,locFichier in enumerate(phrasesFichiers):
    fichier=serie+complementPhrases+locFichier+".csv"
    with codecs.open(fichier, 'r', encoding="utf8") as input:
       exemplesPhrases = input.readlines()
    exemplesPhrases=[exemple.strip().strip(".") for exemple in exemplesPhrases if not exemple.startswith("#")]
    lexComptes, lexPhrases=compterLexique(exemplesPhrases)
    print fichier
    dfLexComptes.append(pd.DataFrame.from_dict(lexComptes,orient="index"))
    dfLexComptes[nFichier].columns=[locFichier]
    dfLexComptes[nFichier]["lexeme"]=dfLexComptes[nFichier].index
    dfLexComptes[nFichier]=dfLexComptes[nFichier][["lexeme",locFichier]]
    if debug:
        print "lexComptes"
        printLexComptes(lexComptes)
        print
        print "lexPhrases"
        printLexPhrases(lexPhrases)
        print
        print
        print

/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/17-K4/Phrases.csv
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/17-K4/Ordre.csv
/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/17-K4/Traductions.csv


In [11]:
#dfLexComptes

In [12]:
df12=dfLexComptes[0].merge(dfLexComptes[1],how="outer").merge(dfLexComptes[2],how="outer")
df12[pd.notnull(df12["Traductions"])].sort_values(by="Ordre")

,lexeme,Phrases,Ordre,Traductions
123,pour,NaN,5.0,3.0
116,quatre.A2,NaN,5.0,7.0
136,chatte.N2.N21.C,NaN,6.0,5.0
99,entrer.VI.V1,NaN,6.0,5.0
133,fruit.N1.N11.D,NaN,7.0,8.0
127,lancer.VT.V2,NaN,7.0,4.0
108,acheter.VD.V2,NaN,7.0,6.0
106,viande.N1.N12.A,NaN,7.0,9.0
104,balai.N2.N21.A,NaN,7.0,8.0
98,offrir.VD.V1,NaN,7.0,5.0


In [287]:
df12.to_csv(path_or_buf=nomComptes,sep="\t",encoding="utf8")

In [288]:
nomComptes

'/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K3/Comptes.csv'